In [1]:
from dubins import connect
import numpy as np
import matplotlib.pyplot as plt
from propagate import propagate
from plot import plot2dcov
from nodes_edges import Edge, BeliefNode
%matplotlib qt

In [17]:
start = [0,1, np.pi]
end = [100,100, np.pi/2]
dt = 0.01
edge =connect(start, end, dt = 0.01, plot = True)

In [18]:
edge_test = Edge(edge[0], edge[1])
belief_test = BeliefNode(np.eye(3)*0.01,np.zeros((3,3)), 0)

In [19]:
belief_output = propagate(edge_test, belief_test, dt)

In [20]:
plot2dcov([0,0], (belief_output.Sigma + belief_output.Lambda)[:2,:2])

In [ ]:
import numpy as np

def AppendBelief(v, new_belief, eps):
    dominated = True
    for node in v.nodes:
        if np.linalg.norm(new_belief.Sigma) < np.linalg.norm(node.Sigma+np.eye(3)*eps) and
        np.linalg.norm(new_belief.Lambda) < np.linalg.norm(node.Lambda+np.eye(3)*eps) and
        new_belief.cost < node.cost:
            dominated = False
    if dominated:
        return False
    else:
        to_delete = []
        for node in v.nodes.copy():
            if np.linalg.norm(new_belief.Sigma) < np.linalg.norm(node.Sigma) and
            np.linalg.norm(new_belief.Lambda) < np.linalg.norm(node.Lambda) and
            new_belief.cost < node.cost:
                v.nodes.remove(node)
    v.nodes.append(new_belief)